<a href="https://colab.research.google.com/github/IIF0403/Thesis/blob/main/OnlyFineTuning(u19).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Check GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
import numpy as np
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as F
from google.colab import files
from google.colab import output
from google.colab import drive
from torch.nn.utils.rnn import pack_sequence
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from sklearn.model_selection import train_test_split
from copy import deepcopy
import random
from datetime import datetime
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#Importing Dataset class from other ipynb file
!pip install import_ipynb
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks'

#!ls
import import_ipynb
from SimSiam_training import *


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks


In [ ]:
### Some helping functions ###

#Function to calculate Accuracy score
def Accuracy_score(labels, preds):
  if len(labels)!=len(preds):
    print("sizes does not match")
  else:
    total=0
    correct=0
    for i in range(len(labels)):
      if labels[i]==preds[i]:
        correct+=1
      total+=1
    return (correct/total)

## Function to calculate classification_accuracy of a classifier given a frozen backbone model the test set
def evaluate_classifier(test_loader, backbone_model, classifier):
  classifier.eval()
  accuracies = []
  for batch in enumerate(test_loader):
    time_series_batch = batch[1]['time_series']
    label_batch = batch[1]['label']

    with torch.no_grad():
      feature = backbone_model(time_series_batch)
      y_hat = classifier(feature)
      pred = torch.max(y_hat,1)[1]
      #print("pred: ", pred)
      #print("true: ", label_batch)
      accuracy = Accuracy_score(label_batch, pred) 
      accuracies.append(accuracy)
  
  Accuracy = np.mean(accuracies)
  return Accuracy

## Function to calculate classification_accuracy of a model (compplete model) given the test set
def evaluate_model(test_loader, model):
  model.eval()
  accuracies = []
  for batch in enumerate(test_loader):
    time_series_batch = batch[1]['time_series']
    label_batch = batch[1]['label']

    with torch.no_grad():
      y_hat = model(time_series_batch)
      pred = torch.max(y_hat,1)[1]
      #print("pred: ", pred)
      #print("true: ", label_batch)
      accuracy = Accuracy_score(label_batch, pred) 
      accuracies.append(accuracy)
  
  Accuracy = np.mean(accuracies)
  return Accuracy


## function to save checkpoint of linear classifier
def save_checkpoint_classifier(SaveName, model, epoch, optimizer, loss_list, accuracy_list, lr, train_bs):
  drive.mount('/content/drive')
  PATH = f"/content/drive/MyDrive/checkpoints/classifier_{SaveName}_{datetime.now().strftime('%d%m')}_ep:{epoch}.pth"
  checkpoint = {'epoch': epoch, 
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss_list': loss_list, 
              'accuracy_list': accuracy_list,
              'lr': lr,
              'train_bs' : train_bs}
  torch.save(checkpoint, PATH)

## function to load checkpoint of linear classifier
def load_checkpoint_classifier(name,date, e):
  drive.mount('/content/drive')
  PATH = f"/content/drive/MyDrive/checkpoints/classifier_{name}_{date}_ep:{e}.pth"
  checkpoint = torch.load(PATH)

  epoch = checkpoint['epoch']
  train_loss = checkpoint['loss_list'][-1]
  max_accuracy = max(checkpoint['accuracy_list'])

  print("checkpoint:", date," Epoch: ", epoch, " Max_accuracy: ",accuracy," Loss: ", train_loss)
  return checkpoint

#Function to print value rounded to 4 desimals 
def rounded(value):
  format = "{:.4f}".format(value)
  float_value = float(format)
  return float_value


#Function to plot Accuracy or Loss over epochs
def plot_progress(res, title):
  N = len(res)
  epochs = [i for i in range(N)]

  plt.figure()
  plt.plot(epochs, res)
  plt.xlabel('epoch')
  plt.title(title)

  plt.savefig(title+".png")
  files.download(title+".png") 
  plt.show()

#### FineTune network model ####
#Combines the Backbone model and the Linear to a complete model for finetuning
class FineTuneModel(nn.Module):
  def __init__(self, Backbone, Linear):
    super(FineTuneModel, self).__init__()
    self.Backbone = Backbone
    self.Linear = Linear
  
  def forward(self, x):
    feature = self.Backbone(x)
    pred = self.Linear(feature)
    return pred


In [ ]:

## Function to do FineTuning 
def FineTuning(trained_model, train_set, test_set, SaveName=None, epochs_Linear = 10, epochs_finetune=10, train_bs = 40, lr_linear=0.001, lr_finetune=0.001, backbone = "FCN"):
  #trained model: a model previously trained with SimSiam
  #train_set, test_set: The datasets to train and evaluate the classification 
  
  dataset = train_set.Datasets[0]
  classes = test_set.classes

  Results = []

  train_loader = DataLoader(train_set, batch_size = train_bs, shuffle=True)
  test_loader = DataLoader(test_set, batch_size = train_bs, shuffle=True)

  
  #Get the frozen backbone of the trained model
  Backbone_model = trained_model.backbone
  Backbone_model = Backbone_model.to(device)
  Backbone_model = nn.DataParallel(Backbone_model)

  #Linear classifier to train in top of the trained frozen model
  lr = lr_linear
  Linear = nn.Linear(128, classes).to(device)
  optimizer_linear = torch.optim.Adam(Linear.parameters(), lr=lr)
  accuracies_linear = []
  losses_linear = []

  ########## Train Linear layer on frozen Backbone ############
  print("Training Linear Layer")
  for e in range(epochs_Linear):
    Backbone_model.eval()
    Linear.train()
    loss_list_linear = []

    for batch in enumerate(train_loader):
      time_series_batch = batch[1]['time_series']
      label_batch = batch[1]['label']

      #Train linear classifier on top of backbone model
      Linear.zero_grad()
      with torch.no_grad():
        feature = Backbone_model(time_series_batch.to(device))
      pred = Linear(feature.to(device))
      loss_linear = F.cross_entropy(pred, label_batch)
      loss_linear.backward()
      optimizer_linear.step()
      loss_list_linear.append(loss_linear.item())
      
    Loss_linear = np.mean(loss_list_linear)
    losses_linear.append(Loss_linear)
    accuracy_linear = evaluate_classifier(test_loader, Backbone_model, Linear)
    accuracies_linear.append(accuracy_linear)

    print("e:  ", e, " dataset: ", dataset,"  SimSiam:   accuracy; ", rounded(accuracy_linear), " loss; ",rounded(Loss_linear))

  results_linear = [dataset, "LinearEval", e , train_bs, lr, accuracies_linear[-1], max(accuracies_linear), losses_linear[-1]]
  Results.append(results_linear)

  #############################################################

  ############### Unfreeze and train the whole network ##################
  print("Fine tuning the whole network")

  #Get a copy of the trained frozen backbone and the trained Linear classifier
  Backbone_finetune = nn.DataParallel(Backbone_model).to(device)
  Linear_finetune = nn.DataParallel(Linear).to(device)

  FineTune_model = FineTuneModel(Backbone_finetune, Linear_finetune) #Combine Backbone and Linear to a single model for finetuning

  lr_FineTune =lr_finetune
  optimizer_FineTune = torch.optim.Adam(FineTune_model.parameters(), lr=lr_FineTune)

  losses_FineTune = []
  accuracies_FineTune = []

  for e in range(epochs_finetune):
    FineTune_model.train()
    loss_list_Finetune = []

    for batch in enumerate(train_loader):
      time_series_batch = batch[1]['time_series']
      label_batch = batch[1]['label']

      #Train the FineTune model
      FineTune_model.zero_grad()
      pred = FineTune_model(time_series_batch.to(device))

      loss_FineTune = F.cross_entropy(pred, label_batch)
      loss_FineTune.backward()
      optimizer_FineTune.step()
      loss_list_Finetune.append(loss_FineTune.item())


    Loss_FineTune = np.mean(loss_list_Finetune)
    losses_FineTune.append(Loss_FineTune)
    accuracy_FineTune = evaluate_model(test_loader, FineTune_model)
    accuracies_FineTune.append(accuracy_FineTune)

    print("e:  ", e, " dataset: ", dataset,"  SimSiam:   accuracy; ",rounded(accuracy_FineTune), " loss; ",rounded(Loss_FineTune))

    if (e in [10,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400]):
      results_finetune = [dataset, "FineTuning", e , train_bs, lr_FineTune, accuracies_FineTune[-1], max(accuracies_FineTune), losses_FineTune[-1]]
      Results.append(results_finetune)

  results_finetune = [dataset, "FineTuning", e , train_bs, lr_FineTune, accuracies_FineTune[-1], max(accuracies_FineTune), losses_FineTune[-1]]
  Results.append(results_finetune)

  #Plotting Accuracy and Loss over epochs
  plot_progress(accuracies_FineTune, "Accuracy_FineTune_"+dataset)
  plot_progress(losses_FineTune, "Loss_FineTune_"+dataset)

  if (SaveName !=None):
    save_checkpoint_classifier(SaveName, FineTuneModel, e, optimizer_FineTune, losses_FineTune, accuracies_FineTune, lr_FineTune, train_bs) #Save checkpoint

  return Results

In [ ]:
##### FineTuning on each dataset of the big dataset #####

#Big_dataset = ["ChlorineConcentration", "Two_Patterns", "yoga", "uWaveGestureLibrary_X", "NonInvasiveFatalECG_Thorax1", "ECG5000", "FordA", "FordB"]
#dat = ["ChlorineConcentration"]

#Load trained model
#Datasets = ["ChlorineConcentration", "ECG5000", "ElectricDevices", "FordA", "FordB", "Two_Patterns", "wafer", "yoga"]
Datasets = ["ChlorineConcentration"]

#SaveNames = ["Window38","Window38_sep","Horizon03_sep" ]
#SaveNames = ["Horizon03_sep"]
#SaveName = "Window38_sep"
#SaveName = "Horizon03_sep"

SaveNames = ["Window38"]
epoch = 99
date = 2704

#SaveNames = ["ResNet_W38"]
#epoch = 99
#date = 2904

backbone = "FCN"
#backbone = "ResNet"

batch_size = 40
#lr = 0.001
lr=None
#labeled_sizes = [0.2,0.1,0.05]
labeled_sizes = [0.1]

rounds = 1
epochs_Linear = 40
epochs_FineTune = 200

lr_linear= 0.01
lr_finetune= 0.05


Results = []

for SaveName in SaveNames:
  checkpoint = load_checkpoint(SaveName, date, epoch)
  trained_model, optimizer_model, epoch_model, lr_model = load_trained_model(checkpoint, backbone=backbone)

  for dataset in Datasets:
    for labeled_size in labeled_sizes:
      Train_set = Timeseries_Dataset([dataset], train=True, Save=None, transform = transforms.Compose( [ ToTensor()] ))
      N_train = len(Train_set)

      Train_set.shuffle()
      Train_set_labeled = Train_set[:int(N_train*labeled_size)]

      Test_set = Timeseries_Dataset([dataset], train=False, Save=None, transform = transforms.Compose( [ ToTensor()] ))

      for round in range(rounds):
        Train_set_labeled.shuffle()
        Test_set.shuffle()
        
        Save_checkpoint_name = None

        Results_1 = FineTuning(trained_model, Train_set_labeled, Test_set, SaveName=Save_checkpoint_name, epochs_Linear = epochs_Linear, epochs_finetune=epochs_FineTune, train_bs = batch_size, lr_linear=lr_linear, lr_finetune=lr_finetune, backbone = backbone)
        
        for res in Results_1:
          result = res
          result.append(SaveName)
          result.append(date)
          result.append(labeled_size)

          Results.append(result)
  
    column_names = ["Dataset", "Type", "epochs", "batch_size", "lr", "last_Acc", "Model Accuracy", "Loss Model", "trained model used", "date model", "labeled size"]    
    to_Excel(Results, column_names, "FineTune_"+SaveName+"_"+dataset+".xlsx")


column_names = ["Dataset", "Type", "epochs", "batch_size", "lr", "last_Acc", "Model Accuracy", "Loss Model", "trained model used", "date model", "labeled size"]

name = "_FineTune_"+SaveName+"_all"
to_Excel(Results, column_names, name+".xlsx")


